In [1]:
import json
import spacy
nlp = spacy.load("en_core_web_sm")

In [204]:
### LOAD ###

#keyphrases = [json.loads(ln)["keyphrase"] for ln in open("keyphrases.jsonl")]
data = [json.loads(ln) for ln in open("../data/train_cmv_cleaned.jsonl")]
#topics = [json.loads(ln) for ln in open("topic_signatures.jsonl")]

In [ ]:
from utils import keyphrase_extraction


In [1]:
# https://aclanthology.org/E17-1024.pdf
# TODOs: Polarity Shifters
# TODOs: Sentence Levels over the entire argument
# MANUAL: Added "least"
from spacy.matcher import PhraseMatcher
import random

phrase_matcher = PhraseMatcher(nlp.vocab)

#id = random.randint(0, 2000)
id = 1586
# keyphrase = nlp(keyphrases[id][0].lower())
#topic = nlp(topics[id]["topic_signature"][0])

claim =  nlp(data[id]["titles"])
arg = nlp(data[id]["argument"])
keyphrase = exctract_keyphrase(claim)

"scripts/lexicon/negative_lex.txt"
# ### SENTIMENT LEXICONS ###
pos = [w.replace("\n", "") for w in open("./lexicon/positive_lex.txt")]
neg = [w.replace("\n", "") for w in open("./lexicon/negative_lex.txt")]

compound_word = ""
for i in keyphrase:
    if i.pos_ in ["NOUN", "PROPN"]:
        comps = "".join([str(j) for j in i.children if j.dep_ == "compound"])
        if comps:
            compound_word = comps + " " + str(i)

# match pattern
patterns = [nlp(compound_word)]
phrase_matcher.add("phrases", None, *patterns)

start = 0
stop = 0
matched_phrases = phrase_matcher(claim)
for i, j, k in matched_phrases:
    start = j
    stop = k

pos_score = 0.0
neg_score = 0.0

# TODOs - Manage Adjectival Complement 'adcomp'
# Sentiment Shifters - reverse polarity of Sentiment Words; k trailing words from Sentiment Shifters

for idx, tok in enumerate(claim):

    if idx == start or idx == stop - 1:
        continue

    # Polarity Shift
    # NEAR parameter, k
    # k = 5
    # if tok.dep_ == "neg":
    #     if tok.text in pos:
    #         # Shift to Negative
    #         if idx <= k:
    #             if idx < start: neg_score += 1/(start - idx)
    #             else: neg_score += 1/(idx - stop)**0.5
    #
    #     if str(tok.head.text) in neg:
    #         # Shift to Positive
    #         if idx < start: pos_score += 1/(start - idx)
    #         else: pos_score += 1/(idx - stop)**0.5

    if str(tok.text) in pos:
        if idx < start: pos_score += 1/(start - idx)
        else: pos_score += 1/(idx - stop)**0.5

    if str(tok.text) in neg:
        if idx < start: neg_score += 1/(start - idx)
        else: neg_score += 1/(idx - stop)**0.5

result = pos_score - neg_score /(pos_score - neg_score + 1)
stance = ""

neg_score, pos_score
stance = {"claim": claim, "stance": "PRO", "aspect": keyphrase, "topic": topic} if result > 0 else {"claim": claim, "stance": "CON", "aspect": keyphrase, "topic": topic}

stance

ModuleNotFoundError: No module named 'keyphrase_extraction'

In [307]:
### CONCEPT NET ###
import requests

obj = requests.get(f"http://api.conceptnet.io/c/en/{stance['aspect']}").json()
obj.keys()

for i in obj["edges"]:
    if i["start"]["language"] == "en":
        print("start", i["start"])
        print("rel", i["rel"])
        print("rel", i["rel"])
        print(i["start"]["label"])
        print("rel", i["rel"]["label"])


start {'@id': '/c/en/retail/v/wn/commerce', '@type': 'Node', 'label': 'retail', 'language': 'en', 'sense_label': 'v, commerce', 'term': '/c/en/retail'}
rel {'@id': '/r/MannerOf', '@type': 'Relation', 'label': 'MannerOf'}
rel {'@id': '/r/MannerOf', '@type': 'Relation', 'label': 'MannerOf'}
retail
rel MannerOf
start {'@id': '/c/en/retail/n/wn/act', '@type': 'Node', 'label': 'retail', 'language': 'en', 'sense_label': 'n, act', 'term': '/c/en/retail'}
rel {'@id': '/r/IsA', '@type': 'Relation', 'label': 'IsA'}
rel {'@id': '/r/IsA', '@type': 'Relation', 'label': 'IsA'}
retail
rel IsA
start {'@id': '/c/en/retail/v/wn/commerce', '@type': 'Node', 'label': 'retail', 'language': 'en', 'sense_label': 'v, commerce', 'term': '/c/en/retail'}
rel {'@id': '/r/HasContext', '@type': 'Relation', 'label': 'HasContext'}
rel {'@id': '/r/HasContext', '@type': 'Relation', 'label': 'HasContext'}
retail
rel HasContext


In [239]:
import graphviz
from graphviz import Source
from nltk import Tree
import deplacy

# sentence = 'The brown fox is quick and he is jumping over the lazy dog'
# doc = nlp(sentence)

dot = graphviz.Digraph(comment='The Round Table')
dot.node('A', 'King Arthur')  # doctest: +NO_EXE
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')

dot.edges(['AB', 'AL'])
dot.edge('B', 'L', constraint='false')

print(dot.source)
dot.render('doctest-output/round-table.gv').replace('\\', '/')

// The Round Table
digraph {
	A [label="King Arthur"]
	B [label="Sir Bedevere the Wise"]
	L [label="Sir Lancelot the Brave"]
	A -> B
	A -> L
	B -> L [constraint=false]
}



'doctest-output/round-table.gv.pdf'

In [255]:
doc = nlp("The quick brown fox jumps over the lazy dog.")

dot = graphviz.Digraph(comment='Test')
def to_tree(node):
    if node.n_lefts + node.n_rights > 0:
        print(node.orth_, node.dep_, node.pos_, [i for i in node.children])
        # dot.edge([to_tree(i) for i in node.children])
        # [to_tree(i) for i in node.children]
    else:node.orth_,
        dot.node(node.orth_)

for sent in doc.sents:
    to_tree(sent.root)

jumps ROOT VERB [fox, over, .]
fox nsubj NOUN [The, quick, brown]


TypeError: edge() missing 1 required positional argument: 'head_name'

In [259]:
import spacy
from nltk import Tree

doc = nlp("its the questions we can't answer that teach us the most.")

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_


[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

                questions                    
  __________________|________________         
 |   |   |          |              teach     
 |   |   |          |           _____|____    
 |   |   |        answer       |     |   most
 |   |   |    ______|______    |     |    |   
its the  .   we     ca    n't that   us  the 



[None]

In [129]:
# doc = nlp("Autonomous cars and magic wands shift insurance liability toward manufacturers")
# interesting_token = doc[7] # or however you identify the token you want
# for noun_chunk in doc.noun_chunks:
#     if interesting_token in noun_chunk:
#         print(noun_chunk)

insurance liability


In [218]:
# for chunk in sent_.noun_chunks:
#     #print(chunk.text)
#     if chunk.text in str(pattern):
#         print(chunk.text, [i for i in chunk.lefts])

Humans are going to die out because of an economic collapse
an economic collapse []


In [ ]:
# ### SENTIMENT LEXICONS ###
# pos = [w.replace("\n", "") for w in open("positive_lex.txt")]
# neg = [w.replace("\n", "") for w in open("negative_lex.txt")]

# id = 2
# claim =  data[id]["titles"] + data[id]["arguments"]
# doc = nlp(claim.lower())
#
# pattern = nlp(keyphrases[id][0].lower())
#
# compound_word = ""
# for i in pattern:
#     if i.pos_ in ["NOUN", "PROPN"]:
#         comps = "".join([str(j) for j in i.children if j.dep_ == "compound"])
#         if comps:
#             compound_word = comps + " " + str(i)
#
#
#
# opinion_sents = []
#
# for i in doc.sents:
#     if str(compound_word) in str(i):
#         opinion_sents.append(i)
#
# stance_distances = {}
# for i, sent in enumerate(opinion_sents):
#     stance_distances[i] = {k: [] for k in sent}
#
#
# # test = [i for i in opinion_sents[0]]
# # for tok in test:
# #     if str(tok.text) in list(str(stance_distances[i].keys())):
# #         print(tok.text)
# #token.text in list(stance_distances[i].keys()):
#
# for i, sent in enumerate(opinion_sents):
#
#     for count, token in enumerate(sent):
#         print(count, token)
#
#         if str(token.text) in list(str(l) for l in stance_distances[i].keys()):
#             pass
#             # if str(token.text) in pos:
#             #     stance_distances[i][token.text].append(("POS", count))
# #
#             # if token.text in neg:
#             #     # stance_distances[i][token.text] = ("NEG", count)
#             #     stance_distances[i][token.text].append(("NEG", count))
#
# #             # else: stance_distances[i][token.text] = ("UNK", count)
# #
#
# print(stance_distances)
#
# # print("benefits" in pos)
# # print(pos)

In [ ]:
# id = 2
# kp = str(" ".join([i for i in keyphrases[id]]))
# kp_doc = nlp(kp)
#
# kp_nouns = []
# kp_nouns.append(" ".join([str(tok) for tok in kp_doc if tok.pos_ in ('NOUN','PROPN')]))
# kp_nouns.append(" ".join(([str(tok) for tok in kp_doc if tok.dep_ == 'compound'])))
#
# kp_nouns = " ".join(kp_nouns)
# print(kp_nouns)
#
# pos = [w for w in open("positive_lex.txt")]
# neg = [w for w in open("negative_lex.txt")]
#
# adj_ = []
# stance = 0
#
# for i in noun_adj_pairs:
#
#     j, k = i
#
#     if j.text in kp_nouns:
#
#         if j.text in pos: stance = 1
#         if j.text in neg: stance = -1
#
#         else: stance = "UNK"
#
#         adj_.append((k, stance))

In [ ]:
# id = 50
# sent_ = data[id]["titles"] + data[id]["arguments"]
# doc = nlp(sent_)
#
# # Consider Childeren == compound etc
# noun_adj_pairs = []
# for i,token in enumerate(doc):
#
#     if token.pos_ not in ('NOUN','PROPN'):
#         continue
#
#     for j in range(i+1,len(doc)):
#         if doc[j].pos_ == 'ADJ' or doc[j].pos_ == "CCONJ":
#             noun_adj_pairs.append((token,doc[j]))
#             break
#
# noun_adj_pairs

In [ ]:
# parsed_sentence = nlp(u'This is my sentence')
# [(token.text,token.i) for token in parsed_sentence]

In [ ]:
# pos = [w.replace("\n", "") for w in open("positive_lex.txt")]
# neg = [w.replace("\n", "") for w in open("negative_lex.txt")]
#
# id = 2
# claim =  data[id]["titles"] + data[id]["arguments"]
# doc = nlp(claim.lower())
#
# pattern = nlp(keyphrases[id][0].lower())
# print(pattern)
#
# compound_word = ""
# for i in pattern:
#     if i.pos_ in ["NOUN", "PROPN"]:
#         comps = "".join([str(j) for j in i.children if j.dep_ == "compound"])
#         if comps:
#             compound_word = comps + " " + str(i)
#
# compound_word

In [ ]:
# Identify the targets of the given topic, claim
# Identify the polarity (sentiment) towards each of the targets



In [132]:
# from spacy.symbols import nsubj, VERB
#
# doc = nlp("Autonomous cars shift insurance liability toward manufacturers")
#
# verbs = set()
# for possible_subject in doc:
#     if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
#         verbs.add(possible_subject.head)

In [ ]:
# OPEN-IE

# IMPORTANT NOTE; Command: java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000
# TODOs: Use openie library, StanfordOpenIE

# import nltk
# from pycorenlp import *
# import json
# import collections
#
# nlp = StanfordCoreNLP("http://localhost:9000/")
# properties = {
#     "annotators": "tokenize, ssplit, pos, depparse, natlog,openie",
#     "outputFormat": "json",
# }
#
# output = []
# sub_pred_obj = []
# titles = []

# for j in data[13:14]:
#     result = json.loads(nlp.annotate(j["titles"], properties=properties))
#     raw = result["sentences"][0]["openie"]
#     if raw:
#         output.append([j["titles"], result["sentences"][0]["openie"]])
#     else:
#         continue
#
# # for i in output:
# #     sub_pred_obj.append({
# #         "title": i[1],
# #         "subject": i[3][0]["subject"],
# #         "predicate": i[3][0]["relation"],
# #         "object": i[3][0]["object"]
# #     })
#
# for i in sub_pred_obj:
#     print(i)

In [250]:
# Nice Implementation

# texts =  ("I believe that the Gay Marriage discussion isnt as important",
#          )
# docs = nlp.pipe(texts)
#
# compounds = []
# for doc in docs:
#     compounds.append({doc.text:[doc[tok.i:tok.head.i+1] for tok in doc if tok.dep_=="compound"]})
# print(compounds)

[{'I believe that the Gay Marriage discussion isnt as important': [Gay Marriage, Marriage discussion]}]
